# AirBnB Reviews Analysis Project

## Table of contents
1. [Introduction](#introduction)
    - [Project Background](#project-background)
    - [Stakeholder Profile](#stakeholder-profile)
2. [Project Objectives](#project-objectives)
    - [Scope of Analysis](#scope-of-analysis)
    - [Data Sources and Collection](#data-sources-and-collection)
3. [Data Preprocessing](#data-preprocessing)
    - [Cleaning and Preparation](#cleaning-and-preparation)
    - [Handling Missing Values](#handling-missing-values)
    - [Normalization and Transformation](#normalization-and-transformation)
4. [Final Dataset Overview](#data-overview)
    - [Dataset Description](#dataset-description)
5. [Text Analytics and Feedback Analysis](#text-analytics-and-feedback-analysis)
    - [Text Preprocessing](#text-preprocessing-tokenization-stop-words-removal-lemmatization)
    - [Sentiment Analysis](#sentiment-analysis)
    - [Topic Extraction](#topic-extraction)


## Introduction

### Project Background

London has established itself as a premier global tourist destination over the past decade, consistently ranking among the top five cities worldwide. The city attracts around 19 million tourists each year, creating a robust tourism market valued at £9.4 billion in 2022. Recognizing this potential, our client, Martin Stuart, is looking to invest in an Airbnb property in London to generate passive income and benefit from property appreciation.

### Stakeholder Profile

Martin Stuart is a well-off, tech-savvy professional with a keen interest in real estate investments, particularly in the Airbnb market. He seeks properties located in tourist-heavy areas with strong rental demand, focusing on family-friendly environments equipped with modern amenities. Martin prefers investments that require minimal renovations and can be easily managed, either personally or through professional services.

## Project Objectives

To assist Martin in making an informed investment decision, we will analyze the Airbnb dataset provided by InsideAirbnb, which includes various property characteristics and host feedback. Our approach involves both tabular data analysis and text analytics to extract meaningful insights from guest comments.

### Scope of Analysis

1. **Property Selection**: We will identify properties based on key characteristics such as:
   - location (approximately 10-15 min walk from top 5 tourist attractions - London eye, Buckingham Palace, Kensington Palace, Big Ben, Kew Gardens)
   - price per night
   - type of property (Entire properties with capacity for 2 or more people)
   - bathrooms and beds (at least 1 of each)
   - number of reviews (properties historically successfull in AirBnB)
We will select the best properties with more comments and higherratings, ensuring that only one property per host is analyzed.

3. **Comment Analysis**: A sample of comments from each selected property will be analyzed using Large Language Models (LLMs) to summarize reviews, detect consistent topics, and identify keywords that contribute to property success or failure. This process will be incremental, starting with sentiment analysis (which properties have a big percentage of negative reviews), summarizing key words in comments to get an overview of common themes (What are lodgers positively most commenting on?)

4. **Data Augmentation**: We will employ Natural Language Processing (NLP) techniques to preprocess the comments, including tokenization, stop words removal, and lemmatization.

5. **Analysis and Recommendations**: Based on our findings, we will perform a comparative analysis of top properties to determine the property characteristics and success. This analysis will provide actionable recommendations for Martin's investment strategy.


### Data Sources and Collection

The data used for this project is sourced from InsideAirbnb, a platform that provides detailed information on Airbnb listings. The dataset includes both listing data and review data, enabling a comprehensive analysis of property characteristics and guest feedback.

- **Listing Data**: This data provides information about each Airbnb listing, including details about the property, host, location, availability, and reviews.
- **Review Data**: This data includes guest reviews, which offer insights into the experiences and satisfaction of guests staying at the listed properties.

Both datasets were last scraped and compiled in 2022, ensuring recent and relevant information for our analysis.



## Data PreProcessing

In [371]:
# import necessary libraries
import pandas as pd
import numpy as np

### Data inspection

In [372]:
# read the listings file
df_listings = pd.read_csv('listings.csv.gz', compression='gzip')

In [373]:
# see info inside df columns
pd.set_option('display.max_columns', None)
df_listings.head()

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms,bathrooms_text,bedrooms,beds,amenities,price,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,312761,https://www.airbnb.com/rooms/312761,20240319050633,2024-03-21,city scrape,Spacious Central London Apartment by Hoxton Sq...,"Very central location, in the middle of Shored...",Everything is so convenient and the area is al...,https://a0.muscache.com/pictures/miso/Hosting-...,1608226,https://www.airbnb.com/users/show/1608226,Pippa,2012-01-15,"London, United Kingdom",I love exploring new places when possible and ...,within a few hours,100%,98%,t,https://a0.muscache.com/im/pictures/user/User-...,https://a0.muscache.com/im/pictures/user/User-...,Shoreditch,1.0,3.0,"['email', 'phone']",t,t,"London, England, United Kingdom",Hackney,NaN,51.52704,-0.08129,Entire rental unit,Entire home/apt,2,1.0,1 bath,1.0,1.0,"[""Toaster"", ""Room-darkening shades"", ""Coffee m...",$160.00,9,180,2.0,9.0,180.0,180.0,7.6,180.0,NaN,t,7,17,30,81,2024-03-21,122,26,0,2012-07-12,2024-02-03,4.96,4.97,4.93,4.96,5.00,4.89,4.93,NaN,f,1,1,0,0,0.86
1,13913,https://www.airbnb.com/rooms/13913,20240319050633,2024-03-20,city scrape,Holiday London DB Room Let-on going,My bright double bedroom with a large window h...,Finsbury Park is a friendly melting pot commun...,https://a0.muscache.com/pictures/miso/Hosting-...,54730,https://www.airbnb.com/users/show/54730,Alina,2009-11-16,"London, United Kingdom",I am a Multi-Media Visual Artist and Creative ...,within a few hours,86%,84%,t,https://a0.muscache.com/im/users/54730/profile...,https://a0.muscache.com/im/users/54730/profile...,LB of Islington,3.0,5.0,"['email', 'phone']",t,t,"Islington, Greater London, United Kingdom",Islington,NaN,51.56861,-0.11270,Private room in rental unit,Private room,1,1.0,1 shared bath,1.0,1.0,"[""Toaster"", ""Room-darkening shades"", ""Shampoo""...",$65.00,1,29,1.0,1.0,29.0,29.0,1.0,29.0,NaN,t,29,52,82,351,2024-03-20,41,1,0,2010-08-18,2023-11-26,4.80,4.73,4.73,4.75,4.83,4.70,4.70,NaN,f,3,2,1,0,0.25
2,15400,https://www.airbnb.com/rooms/15400,20240319050633,2024-03-20,city scrape,Bright Chelsea Apartment. Chelsea!,Lots of windows and light. St Luke's Gardens ...,It is Chelsea.,https://a0.muscache.com/pictures/428392/462d26...,60302,https://www.airbnb.com/users/show/60302,Philippa,2009-12-05,"Royal Borough of Kensington and Chelsea, Unite...","English, grandmother, I have travelled quite ...",within a day,100%,40%,f,https://a0.muscache.com/im/users/60302/profile...,https://a0.muscache.com/im/users/60302/profile...,Chelsea,1.0,1.0,"['email', 'phone']",t,t,"London, United Kingdom",Kensington and Chelsea,NaN,51.48780,-0.16813,Entire rental unit,Entire home/apt,2,1.0,1 bath,1.0,1.0,"[""Shampoo"", ""Wifi"", ""Fire extinguisher"", ""Lugg...",$120.00,4,30,4.0,4.0,30.0,30.0,4.0,30.0,NaN,t,12,12,1

In [374]:
# Data inspection
df_listings.head()

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms,bathrooms_text,bedrooms,beds,amenities,price,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,312761,https://www.airbnb.com/rooms/312761,20240319050633,2024-03-21,city scrape,Spacious Central London Apartment by Hoxton Sq...,"Very central location, in the middle of Shored...",Everything is so convenient and the area is al...,https://a0.muscache.com/pictures/miso/Hosting-...,1608226,https://www.airbnb.com/users/show/1608226,Pippa,2012-01-15,"London, United Kingdom",I love exploring new places when possible and ...,within a few hours,100%,98%,t,https://a0.muscache.com/im/pictures/user/User-...,https://a0.muscache.com/im/pictures/user/User-...,Shoreditch,1.0,3.0,"['email', 'phone']",t,t,"London, England, United Kingdom",Hackney,NaN,51.52704,-0.08129,Entire rental unit,Entire home/apt,2,1.0,1 bath,1.0,1.0,"[""Toaster"", ""Room-darkening shades"", ""Coffee m...",$160.00,9,180,2.0,9.0,180.0,180.0,7.6,180.0,NaN,t,7,17,30,81,2024-03-21,122,26,0,2012-07-12,2024-02-03,4.96,4.97,4.93,4.96,5.00,4.89,4.93,NaN,f,1,1,0,0,0.86
1,13913,https://www.airbnb.com/rooms/13913,20240319050633,2024-03-20,city scrape,Holiday London DB Room Let-on going,My bright double bedroom with a large window h...,Finsbury Park is a friendly melting pot commun...,https://a0.muscache.com/pictures/miso/Hosting-...,54730,https://www.airbnb.com/users/show/54730,Alina,2009-11-16,"London, United Kingdom",I am a Multi-Media Visual Artist and Creative ...,within a few hours,86%,84%,t,https://a0.muscache.com/im/users/54730/profile...,https://a0.muscache.com/im/users/54730/profile...,LB of Islington,3.0,5.0,"['email', 'phone']",t,t,"Islington, Greater London, United Kingdom",Islington,NaN,51.56861,-0.11270,Private room in rental unit,Private room,1,1.0,1 shared bath,1.0,1.0,"[""Toaster"", ""Room-darkening shades"", ""Shampoo""...",$65.00,1,29,1.0,1.0,29.0,29.0,1.0,29.0,NaN,t,29,52,82,351,2024-03-20,41,1,0,2010-08-18,2023-11-26,4.80,4.73,4.73,4.75,4.83,4.70,4.70,NaN,f,3,2,1,0,0.25
2,15400,https://www.airbnb.com/rooms/15400,20240319050633,2024-03-20,city scrape,Bright Chelsea Apartment. Chelsea!,Lots of windows and light. St Luke's Gardens ...,It is Chelsea.,https://a0.muscache.com/pictures/428392/462d26...,60302,https://www.airbnb.com/users/show/60302,Philippa,2009-12-05,"Royal Borough of Kensington and Chelsea, Unite...","English, grandmother, I have travelled quite ...",within a day,100%,40%,f,https://a0.muscache.com/im/users/60302/profile...,https://a0.muscache.com/im/users/60302/profile...,Chelsea,1.0,1.0,"['email', 'phone']",t,t,"London, United Kingdom",Kensington and Chelsea,NaN,51.48780,-0.16813,Entire rental unit,Entire home/apt,2,1.0,1 bath,1.0,1.0,"[""Shampoo"", ""Wifi"", ""Fire extinguisher"", ""Lugg...",$120.00,4,30,4.0,4.0,30.0,30.0,4.0,30.0,NaN,t,12,12,1

In [375]:
# Data description
df_listings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90852 entries, 0 to 90851
Data columns (total 75 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            90852 non-null  int64  
 1   listing_url                                   90852 non-null  object 
 2   scrape_id                                     90852 non-null  int64  
 3   last_scraped                                  90852 non-null  object 
 4   source                                        90852 non-null  object 
 5   name                                          90852 non-null  object 
 6   description                                   87851 non-null  object 
 7   neighborhood_overview                         47521 non-null  object 
 8   picture_url                                   90842 non-null  object 
 9   host_id                                       90852 non-null 

#### Remove empty columns

In [376]:
# Remove empty columns
df_listings.dropna(axis=1, how='all', inplace=True)
df_listings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90852 entries, 0 to 90851
Data columns (total 72 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            90852 non-null  int64  
 1   listing_url                                   90852 non-null  object 
 2   scrape_id                                     90852 non-null  int64  
 3   last_scraped                                  90852 non-null  object 
 4   source                                        90852 non-null  object 
 5   name                                          90852 non-null  object 
 6   description                                   87851 non-null  object 
 7   neighborhood_overview                         47521 non-null  object 
 8   picture_url                                   90842 non-null  object 
 9   host_id                                       90852 non-null 

#### Remove unwanted columns

In [377]:
# remove unwanted columns
# List of columns to remove
columns_to_remove = [ 'scrape_id','last_scraped','source','picture_url','bedrooms'
                    , 'minimum_nights', 'minimum_minimum_nights',
                     'maximum_minimum_nights','minimum_maximum_nights','maximum_maximum_nights',
                     'minimum_nights_avg_ntm','maximum_nights_avg_ntm','has_availability','availability_30',
                    'availability_60','availability_90','availability_365','calendar_last_scraped',
                     'first_review','last_review','instant_bookable','calculated_host_listings_count_entire_homes',
                     'calculated_host_listings_count_private_rooms','calculated_host_listings_count_shared_rooms',
                     'neighbourhood','neighborhood_overview', 'room_type', 'review_scores_accuracy',
                     'number_of_reviews_ltm', 'number_of_reviews_l30d','maximum_nights'
                    ]

In [378]:
# Drop the specified columns from the original DataFrame
df_listings = df_listings.drop(columns=columns_to_remove)

#### Filter by touristy location latitude and longitude

The top 5 London hallmarks used are:
- London eye
- Buckingham Palace
- Hyde Park
- Big Ben
- Kew Gardens

the latitude and longitude limits from these areas are:
- [51.4965, 51.5150] [-0.1975, -0.1148] for London eye, Buckingham Palace, Hyde Park, Big Ben
- [51.4878, 51.4611] [-0.3271, -0.2823] for Kew Gardens

In [379]:
#Re using the function to filter boundaries used in the python data analysis project
# Function to filter DataFrame based on city boundaries
def filter_by_bounds(df, bounds):
    """
    Filters the input DataFrame based on latitude and longitude boundaries.

    Parameters:
    df (pandas.DataFrame): The input DataFrame containing 'Latitude' and 'Longitude' columns.
    bounds (dict): A dictionary with the following keys:
        - 'min_lat': Minimum latitude boundary
        - 'max_lat': Maximum latitude boundary
        - 'min_lon': Minimum longitude boundary
        - 'max_lon': Maximum longitude boundary

    Returns:
    pandas.DataFrame: A DataFrame containing only the rows where 'Latitude' and 'Longitude'
                      are within the specified bounds.
    """
    return df[
        (df['latitude'] >= bounds['min_lat']) &  # Check if Latitude is greater than or equal to min_lat
        (df['latitude'] <= bounds['max_lat']) &  # Check if Latitude is less than or equal to max_lat
        (df['longitude'] >= bounds['min_lon']) &  # Check if Longitude is greater than or equal to min_lon
        (df['longitude'] <= bounds['max_lon'])    # Check if Longitude is less than or equal to max_lon
    ]

In [380]:
# These boundaries are defined as dictionaries with keys for min and max latitude and longitude
area1_bounds = {'min_lat': 51.4965, 'max_lat': 51.5150, 'min_lon': -0.1975, 'max_lon': -0.1148}
area2_bounds = {'min_lat': 51.4611, 'max_lat': 51.4878, 'min_lon': -0.3271, 'max_lon': -0.2823}

In [381]:
# Apply the filter_by_bounds function to combined_data for each city's boundaries
# Filter the listings for each area
df_area1 = filter_by_bounds(df_listings, area1_bounds)
df_area2 = filter_by_bounds(df_listings, area2_bounds)

In [382]:
# Combine the filtered DataFrames
df_listings = pd.concat([df_area1, df_area2]).drop_duplicates().reset_index(drop=True)

In [383]:
df_listings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5310 entries, 0 to 5309
Data columns (total 41 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              5310 non-null   int64  
 1   listing_url                     5310 non-null   object 
 2   name                            5310 non-null   object 
 3   description                     5122 non-null   object 
 4   host_id                         5310 non-null   int64  
 5   host_url                        5310 non-null   object 
 6   host_name                       5310 non-null   object 
 7   host_since                      5310 non-null   object 
 8   host_location                   3756 non-null   object 
 9   host_about                      3069 non-null   object 
 10  host_response_time              4381 non-null   object 
 11  host_response_rate              4381 non-null   object 
 12  host_acceptance_rate            45

#### Remove rows without a price per night
The 'price' column in essential for the analysis and to give the stakeholder the best recomendations on properties. Therefore no rows with nulls in this column can exist for the analysis.

In [384]:
# Filter the DataFrame to show only rows where 'price' is not null
df_listings = df_listings[df_listings['price'].notna()]

# Changing 'price' format from string to float
# Remove non-numeric characters and convert to float
df_listings['price'] = df_listings['price'].replace('[\$,]', '', regex=True).astype(float)

# Display the filtered DataFrame
df_listings.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4101 entries, 0 to 5309
Data columns (total 41 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              4101 non-null   int64  
 1   listing_url                     4101 non-null   object 
 2   name                            4101 non-null   object 
 3   description                     3980 non-null   object 
 4   host_id                         4101 non-null   int64  
 5   host_url                        4101 non-null   object 
 6   host_name                       4101 non-null   object 
 7   host_since                      4101 non-null   object 
 8   host_location                   2852 non-null   object 
 9   host_about                      2461 non-null   object 
 10  host_response_time              3889 non-null   object 
 11  host_response_rate              3889 non-null   object 
 12  host_acceptance_rate            3982 no

#### Check uniqueness of listing ID

In [385]:
# Check uniqueness of ID
df_listings['id'].nunique() # 4101 so all id's are unique

4101

#### Filtering property type

In [386]:
df_listings['property_type'].unique()

array(['Entire rental unit', 'Private room in rental unit',
       'Entire condo', 'Private room in condo', 'Entire townhouse',
       'Private room in home', 'Entire loft', 'Entire home',
       'Private room in townhouse', 'Room in bed and breakfast',
       'Private room in bed and breakfast', 'Room in aparthotel',
       'Entire serviced apartment', 'Room in serviced apartment',
       'Private room in hostel', 'Entire villa', 'Room in hotel',
       'Room in boutique hotel', 'Shared room in hostel',
       'Entire cottage', 'Private room in guesthouse',
       'Private room in serviced apartment', 'Entire vacation home',
       'Entire home/apt', 'Entire guesthouse', 'Castle', 'Entire place',
       'Private room in casa particular', 'Shared room in rental unit',
       'Private room in guest suite', 'Casa particular',
       'Entire guest suite', 'Shared room in hotel',
       'Shared room in tent', 'Boat', 'Houseboat'], dtype=object)

Considering the way 'property_type' is save dis not coherent, the columns will be separated for twhether the listing represent an entire property or  simply a room.
This preprocessing removed rooms for rental and gave only entire properties that the stakeholder can invest on to put to rent.

In [387]:
# Define a function to split the string appropriately
def split_type_location(description):
    if not description or not isinstance(description, str):
        return ['entire', '']
    parts = description.split(' in ', 1)
    if len(parts) == 2:
        return parts
    else:
        return ['entire', description]

# Ensure no NaN values in 'property_type'
df_listings['property_type'] = df_listings['property_type'].fillna('')

# Apply the function to the 'property_type' column
split_data = df_listings['property_type'].apply(split_type_location).tolist()

# Create a DataFrame from the split data
df_split = pd.DataFrame(split_data, columns=['Type', 'property'])

# Ensure the indices are aligned correctly
df_listings = df_listings.reset_index(drop=True)
df_split = df_split.reset_index(drop=True)

# Concatenate the new DataFrame with the original DataFrame
df_listings = pd.concat([df_listings, df_split], axis=1)

In [388]:
# Remove 'Entire' from column 'property'
df_listings['property'] = df_listings['property'].str.replace('entire ', '')

# Convert Type and Location columns to lowercase
df_listings = df_listings.applymap(lambda x: x.lower() if isinstance(x, str) else x)

C:\Users\MartaMoreira\AppData\Local\Temp\ipykernel_18368\2614879464.py:5: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_listings = df_listings.applymap(lambda x: x.lower() if isinstance(x, str) else x)


In [389]:
# Check if the separation was done properly
df_listings.loc[:, ['property_type','Type','property']].sample(10)

,property_type,Type,property
2458,entire rental unit,entire,entire rental unit
894,private room in rental unit,private room,rental unit
3328,entire rental unit,entire,entire rental unit
2360,entire rental unit,entire,entire rental unit
1960,entire villa,entire,entire villa
3696,entire rental unit,entire,entire rental unit
2614,entire rental unit,entire,entire rental unit
2810,entire rental unit,entire,entire rental unit
3991,entire home,entire,entire home
3977,entire home,entire,entire home


In [390]:
df_listings['Type'].unique()

array(['entire', 'private room', 'room', 'shared room'], dtype=object)

In [391]:
# Filter and keep only rows where Type starts with 'Entire'
df_listings = df_listings[df_listings['Type'].str.startswith('entire')]

In [392]:
df_listings['property'].unique()

array(['entire rental unit', 'entire condo', 'entire townhouse',
       'entire loft', 'entire home', 'entire serviced apartment',
       'entire villa', 'entire cottage', 'entire vacation home',
       'entire home/apt', 'entire guesthouse', 'castle', 'entire place',
       'casa particular', 'entire guest suite', 'boat', 'houseboat'],
      dtype=object)

In [393]:
# Remove unwanted rows according to the stakeholders needs
# List of descriptions to remove
descriptions_to_remove = ['guest suite', 'guesthouse', 'camper/rv', 'boat', 'private room',
                          'shepherd’s hut', 'houseboat', 'farm stay', 'treehouse', 'campsite',
                          'shipping container', 'religious building', 'shared room', 'riad',
                          'island', 'minsu', 'hut', 'dome', 'tent']

# Create a regex pattern to match any of the descriptions
pattern = '|'.join(descriptions_to_remove)

# Filter rows where Location does not contain any of the descriptions
df_listings = df_listings[~df_listings['property'].str.contains(pattern, case=False)]


In [394]:
df_listings['property'].unique()

array(['entire rental unit', 'entire condo', 'entire townhouse',
       'entire loft', 'entire home', 'entire serviced apartment',
       'entire villa', 'entire cottage', 'entire vacation home',
       'entire home/apt', 'castle', 'entire place', 'casa particular'],
      dtype=object)

#### Removing listing with 0 or below average number of reviews

In [395]:
# remove number of reviews =0, (calculate average number of reviews then remove all properties under the average)

# Remove rows where number_of_reviews is equal to 0 or null
df_listings = df_listings[(df_listings['number_of_reviews'] != 0) & (~df_listings['number_of_reviews'].isnull())]

'''# Calculate the average number of reviews
average_reviews = df_listings['number_of_reviews'].mean()

# Filter out properties with number of reviews less than the average
df_listings = df_listings[df_listings['number_of_reviews'] >= average_reviews]'''

"# Calculate the average number of reviews\naverage_reviews = df_listings['number_of_reviews'].mean()\n\n# Filter out properties with number of reviews less than the average\ndf_listings = df_listings[df_listings['number_of_reviews'] >= average_reviews]"

#### Removing 0 beds and 0 bathrooms properties

In [396]:
df_listings['bathrooms_text'].unique()

array(['1 bath', '3 baths', '2 baths', '1.5 baths', '4.5 baths',
       '2.5 baths', '4 baths', '6 baths', '5 baths', '3.5 baths',
       '0 baths', '6.5 baths', '7 baths', '5.5 baths', 'half-bath'],
      dtype=object)

In [397]:
# Remove rows where bathrooms_number is 0
df_listings = df_listings[df_listings['bathrooms'] != 0]

In [398]:
df_listings['bathrooms'].unique()

array([1. , 3. , 2. , 1.5, 4.5, 2.5, 4. , 6. , 5. , 3.5, 6.5, 7. , 5.5,
       0.5])

In [399]:
# Remove rows where the 'beds' column has a value of 0
df_listings = df_listings[df_listings['beds'] != 0]

In [400]:
df_listings.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2497 entries, 1 to 4089
Data columns (total 43 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              2497 non-null   int64  
 1   listing_url                     2497 non-null   object 
 2   name                            2497 non-null   object 
 3   description                     2424 non-null   object 
 4   host_id                         2497 non-null   int64  
 5   host_url                        2497 non-null   object 
 6   host_name                       2497 non-null   object 
 7   host_since                      2497 non-null   object 
 8   host_location                   1844 non-null   object 
 9   host_about                      1641 non-null   object 
 10  host_response_time              2420 non-null   object 
 11  host_response_rate              2420 non-null   object 
 12  host_acceptance_rate            2471 no

#### Remove properties that are just for 1 person

The stakeholder want the property to be family friendly. Thereforefore it's necessary that the property has the space for more than 1 person.

In [401]:
# Filter rows where 'accommodates' is greater than or equal to 2
df_listings = df_listings[df_listings['accommodates'] >= 2]

#### Check amenities

Martin requires the property to have modern amenities but the amenities column is not consistent. Therefore, we decided to check uniquely for wifi as it is an external add on to the property.

In [402]:
# Filter rows where 'wifi' is in the 'amenities' column
df_listings = df_listings[df_listings['amenities'].apply(lambda amenities: 'wifi' in amenities)]

In [403]:
# Display rows where 'wifi' is in the amenities
df_listings.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2443 entries, 1 to 4089
Data columns (total 43 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              2443 non-null   int64  
 1   listing_url                     2443 non-null   object 
 2   name                            2443 non-null   object 
 3   description                     2370 non-null   object 
 4   host_id                         2443 non-null   int64  
 5   host_url                        2443 non-null   object 
 6   host_name                       2443 non-null   object 
 7   host_since                      2443 non-null   object 
 8   host_location                   1808 non-null   object 
 9   host_about                      1613 non-null   object 
 10  host_response_time              2368 non-null   object 
 11  host_response_rate              2368 non-null   object 
 12  host_acceptance_rate            2418 no

#### Host information
Remove extra information on the host and save it in a diffrent csv file in case the stakeholder wants more information from the host.

In [404]:
# Remove information on the host to have separatly in case it's needed
# List of columns to remove
columns_host = [ 'host_id','host_url','host_name','host_since','host_location',
                     'host_about','host_response_time','host_response_rate','host_acceptance_rate',
                     'host_is_superhost', 'host_thumbnail_url','host_picture_url','host_neighbourhood',
                    'host_listings_count','host_total_listings_count', 'host_verifications',
                     'host_has_profile_pic', 'host_identity_verified']

In [405]:
# Select the columns to remove and create a new DataFrame
dropped_columns_df = df_listings[columns_host]
# Remove 'host_id' from the list
columns_host.remove('host_id')
# Drop the specified columns from the original DataFrame
df_listings = df_listings.drop(columns=columns_host)

In [406]:
# Save the DataFrame with the dropped columns to a separate CSV file
dropped_columns_df.to_csv('host_info.csv', index=False)

It's also important to take into consideration that the stakeholder only wants a single property per host.

In [407]:
idx = df_listings.groupby('host_id')['number_of_reviews'].idxmax()
df_listings = df_listings.loc[idx].reset_index(drop=True)

In [408]:
df_listings['host_id'].nunique()

1017

#### Remove extra unwanted columns

In [409]:
# Drop the specified columns from the original DataFrame
df_listings = df_listings.drop(columns=['property_type','Type','bathrooms_text'])

#### Save information

In [410]:
#Reset index
df_listings = df_listings.reset_index(drop=True)

In [411]:
# save it into a csv file
df_listings.to_csv('listings_clean.csv', index=False)

### Reviews dataset

In [412]:
# read the listings file
df_reviews = pd.read_csv('reviews.csv.gz', compression='gzip')

In [413]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1618347 entries, 0 to 1618346
Data columns (total 6 columns):
 #   Column         Non-Null Count    Dtype 
---  ------         --------------    ----- 
 0   listing_id     1618347 non-null  int64 
 1   id             1618347 non-null  int64 
 2   date           1618347 non-null  object
 3   reviewer_id    1618347 non-null  int64 
 4   reviewer_name  1618347 non-null  object
 5   comments       1618189 non-null  object
dtypes: int64(3), object(3)
memory usage: 74.1+ MB


#### Only reviews for the properties being analysed

In [414]:
# keep reviews only for the propreties being analysed

# Get the set of ids from df_listings
valid_ids = set(df_listings['id'])

# Filter df_reviews based on valid ids
filtered_reviews = df_reviews[df_reviews['listing_id'].isin(valid_ids)]

# Update df_reviews with the filtered results
df_reviews = filtered_reviews.copy()

In [415]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Index: 35320 entries, 11214 to 1618320
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   listing_id     35320 non-null  int64 
 1   id             35320 non-null  int64 
 2   date           35320 non-null  object
 3   reviewer_id    35320 non-null  int64 
 4   reviewer_name  35320 non-null  object
 5   comments       35315 non-null  object
dtypes: int64(3), object(3)
memory usage: 1.9+ MB


#### Top properties with most reviews in the past 6 months

In [416]:
#number of reviews in the past 6 months
from datetime import datetime, timedelta

# Ensure the 'date' column is in datetime format
df_reviews['date'] = pd.to_datetime(df_reviews['date'])

# Calculate the date 6 months ago from today
six_months_ago = datetime.now() - timedelta(days=6*30)  # Roughly 6 months ago

# Filter the reviews made in the past 6 months
df_reviews = df_reviews[df_reviews['date'] >= six_months_ago]

# Optionally, print the result to verify
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2879 entries, 20258 to 1618320
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   listing_id     2879 non-null   int64         
 1   id             2879 non-null   int64         
 2   date           2879 non-null   datetime64[ns]
 3   reviewer_id    2879 non-null   int64         
 4   reviewer_name  2879 non-null   object        
 5   comments       2878 non-null   object        
dtypes: datetime64[ns](1), int64(3), object(2)
memory usage: 157.4+ KB


In [417]:
df_reviews['listing_id'].nunique() # 687 unique properties with 2906 reviews in the past 6 months

686

### Properties with the highest number of reviews (in the past 6 months)

In [418]:
# Group by 'listing_id' and count non-null comments
comment_counts = df_reviews.groupby('listing_id')['comments'].count().reset_index(name='num_comments')

# Sort by number of comments in descending order and get top 10 listings
top_20_properties = comment_counts.sort_values(by='num_comments', ascending=False).head(20)

# Merge top 10 properties with full listings information
top_20_listings_info = pd.merge(top_20_properties, df_reviews, on='listing_id', how='left')

In [419]:
display(top_20_listings_info)

,listing_id,num_comments,id,date,reviewer_id,reviewer_name,comments
0,51496964,25,1055785757221992726,2023-12-27,506769554,Nicole,Consiglio questo alloggio per chiunque voglia ...
1,51496964,25,1058676765165726760,2023-12-31,203280274,Caroline,Excellent place to stay! Dom’s place was clean...
2,51496964,25,1069543386137174350,2024-01-15,167990729,Michelle,An amazing top floor apartment right in the ce...
3,51496964,25,1070212971517502439,2024-01-16,77179117,Eve,Loved our stay here for three days in London. ...
4,51496964,25,1070980225048496049,2024-01-17,76574585,Giuseppe,"appartamento davvero pulito, luminoso e in una..."
...,...,...,...,...,...,...,...
362,877994002042801699,15,1095590990824223053,2024-02-20,408330886,Kristine,Amalia’s appartment was perfect for our family...
363,877994002042801699,15,1097094738002880169,2024-02-22,55000635,Geoffroy,Très bon sejour chez Amalia. Appartement extrê...
364,877994002042801699,15,1103599704472518350,2024-03-02,277868623,Malak,Amalia was very welcoming and showed us the pl...
365,877994002042801699,15,1107226215343985154,2024-03-07,387037214,Stephen,"Great Place, great location, Clean and well ma..."


In [420]:
top_20_listings_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 367 entries, 0 to 366
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   listing_id     367 non-null    int64         
 1   num_comments   367 non-null    int64         
 2   id             367 non-null    int64         
 3   date           367 non-null    datetime64[ns]
 4   reviewer_id    367 non-null    int64         
 5   reviewer_name  367 non-null    object        
 6   comments       367 non-null    object        
dtypes: datetime64[ns](1), int64(4), object(2)
memory usage: 20.2+ KB


## Final Dataset Overview

### Dataset Description

The Airbnb dataset provided by InsideAirbnb contains a comprehensive range of features related to Airbnb listings in London. The dataset includes the following columns:

- **id** (int64): Unique identifier for the listing
- **listing_url** (object): URL of the listing
- **name** (object): Name of the listing
- **description** (object): Description of the listing
- **host_id** (int64): Unique identifier for the host
- **neighbourhood_cleansed** (object): Cleansed neighborhood name
- **latitude** (float64): Latitude of the listing
- **longitude** (float64): Longitude of the listing
- **property_type** (object): Type of property
- **accommodates** (int64): Number of people the listing accommodates
- **bathrooms** (float64): Number of bathrooms
- **bedrooms** (float64): Number of bedrooms
- **beds** (float64): Number of beds
- **amenities** (object): Amenities available in the listing
- **price** (object): Price per night
- **number_of_reviews** (int64): Total number of reviews
- **review_scores_rating** (float64): Overall rating score
- **review_scores_accuracy** (float64): Rating for accuracy
- **review_scores_cleanliness** (float64): Rating for cleanliness
- **review_scores_checkin** (float64): Rating for check-in experience
- **review_scores_communication** (float64): Rating for communication
- **review_scores_location** (float64): Rating for location
- **review_scores_value** (float64): Rating for value
- **reviews_per_month** (float64): Number of reviews per month

Additionally, the dataset for the reviews includes the following columns:

- **listing_id** (int64): Identifier for the listing
- **id** (int64): Identifier for the review
- **date** (object): Date of the review
- **reviewer_id** (int64): Identifier for the reviewer
- **reviewer_name** (object): Name of the reviewer
- **comments** (object): Review comments

### Text Analytics and Feedback Analysis

### Importing libraries and defining functions

In [283]:
import openai
import os
##Note the best practice, in the future try to setup an Env variable. 
apikey = "API-KEY"
client = openai.OpenAI(api_key = apikey)

In [327]:
##This function will return the full output of the API call:

def get_completion(prompt, model="gpt-3.5-turbo", temperature=0):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature = temperature
    )
    return response.choices[0].message.content

# prompt - question
# model - model giving the inputs to
# temperature - parameter that allows the model to be more creative (1- answers more different, 0 - answers quite similar)
# message - always a list with a dict in it. role - user or agent, content - what you're sending
# response - api call

In [421]:
list_2 =[]
for comment in top_20_listings_info['comments']:
    
    # Example: Assuming the API response indicates sentiment as 'positive' or 'negative'
    prompt = f""" Analyze the comment and translate if not in english: '{comment}'
    Perform the following actions:
    1 - List the 3 main topics mentioned, prioritizing 'family friendly' if mentioned. Maximun of three words per topic.
    2 - Analyze the sentiment (positive, negative or neutral).
    
    Output only the answers separated by '\n' """
    
    response = get_completion(prompt)
    list_2.append(response)
    


KeyboardInterrupt



In [422]:
print(len(list_2))

11


In [330]:
# Initialize empty lists to collect data
topics = []
sentiments = []

# Process each comment
for comment in list_2:
    parts = comment.split('\n')  # Split comment into parts by newline
    if len(parts) == 2:
        topic_part = parts[0].split(' - ')
        sentiment_part = parts[1].split(' - ')
        if len(topic_part) == 2 and len(sentiment_part) == 2:
            topics.append(topic_part[1])  # Extract main topics
            sentiments.append(sentiment_part[1])  # Extract sentiment
        else:
            topics.append('')  # Handle cases where topics are not found
            sentiments.append('')  # Handle cases where sentiment is not found
    else:
        topics.append('')  # Handle cases where comment structure is unexpected
        sentiments.append('')  # Handle cases where comment structure is unexpected

# Create DataFrame with extracted data
df_new = pd.DataFrame({
    'topics': topics,
    'sentiment': sentiments
})

# Concatenate df_reviews and df_new along columns axis (axis=1)
df_reviews = pd.concat([top_20_listings_info, df_new], axis=1)

display(df_reviews)

,listing_id,num_comments,id,date,reviewer_id,reviewer_name,comments,topics,sentiment
0,51496964,25,1055785757221992726,2023-12-27,506769554,Nicole,Consiglio questo alloggio per chiunque voglia ...,"Location, Cleanliness, Amenities",Positive
1,51496964,25,1058676765165726760,2023-12-31,203280274,Caroline,Excellent place to stay! Dom’s place was clean...,"Clean, comfortable, location",Positive
2,51496964,25,1069543386137174350,2024-01-15,167990729,Michelle,An amazing top floor apartment right in the ce...,"Location, Amenities, Check-in",Positive
3,51496964,25,1070212971517502439,2024-01-16,77179117,Eve,Loved our stay here for three days in London. ...,"Location, Amenities, Host",Positive
4,51496964,25,1070980225048496049,2024-01-17,76574585,Giuseppe,"appartamento davvero pulito, luminoso e in una...","clean, bright, location",positive
...,...,...,...,...,...,...,...,...,...
365,959893956899055246,15,1095645498070774307,2024-02-20,24459590,Rai,I’ve stayed in London several times in the pas...,"Location, Apartment, Host",Positive
366,959893956899055246,15,1099203608918324845,2024-02-25,62983740,Gabrielle,Steve’s place was just what we needed. The loc...,"Location, Accessibility, Entertainment",Positive
367,959893956899055246,15,1101409329332586088,2024-02-28,218420134,Jessica,Another excellent stay. This has fast become m...,"Excellent stay, favorite airbnb, London",Positive
368,959893956899055246,15,1110054180139338227,2024-03-11,501698457,Sam,Perfect location and great hosts. Thank you!,"Location, hosts, perfect",Positive


In [331]:
#df_reviews.to_csv('top10properties.csv', index=False)

In [341]:
# Function to count topics within each group
def count_topics(group):
    topics_series = group['topics'].str.split(', ').explode()
    return topics_series.value_counts()
    
df_reviews['topics'] = df_reviews['topics'].str.lower()
df_reviews['sentiment'] = df_reviews['sentiment'].str.lower()

# Group by listing_id and apply count_topics function
grouped_topics = df_reviews.groupby('listing_id').apply(count_topics)


In [342]:
df_reviews['listing_id'].unique()

array([           51496964,  872950232509198703,  946222306868694414,
                  13572875,            30141705,            45874669,
                  19587017,  846834270087464008,            53354926,
                  20399917, 1057865236158582573,            14300513,
        962127121469363434,  971811518060382945,  935332067081342415,
                  25175483,            31645964,  877994002042801699,
        943817677972185100,  959893956899055246], dtype=int64)

In [343]:
grouped_topics

listing_id           topics         
13572875             location           11
                     family friendly     4
                     restaurants         3
                     london              2
                     accommodation       2
                                        ..
1057865236158582573  repeat stay         1
                     cozy                1
                     kitchen items       1
                     space               1
                     affordable          1
Name: count, Length: 668, dtype: int64

#### Main topics in positive comments

In [344]:
# narrow down properties that have 'family friendly' as a topic

# Filter for positive sentiment comments
positive_comments = df_reviews[df_reviews['sentiment'] == 'positive']

# Group by listing_id and apply count_topics function
grouped_topics = positive_comments.groupby('listing_id').apply(count_topics).reset_index()

# Rename columns for clarity
grouped_topics.columns = ['listing_id', 'topic', 'count']

# Filter for 'family friendly' topic
family_friendly_counts = grouped_topics[grouped_topics['topic'] == 'family friendly']

# Sort by count descending and select top 5 listings
top_5_properties = family_friendly_counts.sort_values(by='count', ascending=False).head(5)


In [345]:
display(top_5_properties)

,listing_id,topic,count
332,872950232509198703,family friendly,5
370,877994002042801699,family friendly,4
161,30141705,family friendly,3
502,962127121469363434,family friendly,2
477,959893956899055246,family friendly,2


#### Main topics in negative comments

In [346]:
# Filter for negative sentiment comments
negative_comments = df_reviews[df_reviews['sentiment'] == 'negative']

# Split and explode the topics column to count individual topics
negative_comments['topics'] = negative_comments['topics'].str.split(', ')
exploded_topics = negative_comments.explode('topics')

# Count occurrences of each topic
topic_counts = exploded_topics['topics'].value_counts().reset_index()

# Rename columns for clarity
topic_counts.columns = ['topic', 'count']

# Sort by count descending to identify the main topics
sorted_topic_counts = topic_counts.sort_values(by='count', ascending=False).head(5)

'''# Group by listing_id and topic, and count occurrences
grouped_topics = exploded_topics.groupby(['listing_id', 'topics']).size().reset_index(name='count')

# Sort by listing_id and count descending to identify the main topics for each listing
sorted_grouped_topics = grouped_topics.sort_values(by=['listing_id', 'count'], ascending=[True, False])
'''
#print("Most common topics in negative comments:")
display(sorted_grouped_topics)

C:\Users\MartaMoreira\AppData\Local\Temp\ipykernel_18368\3724262621.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  negative_comments['topics'] = negative_comments['topics'].str.split(', ')


,listing_id,topics,count
0,20399917,accommodation,1
1,20399917,amenities,1
2,20399917,cleanliness,1
3,20399917,location,1
4,20399917,london,1
5,20399917,noise,1
6,30141705,comfort,1
7,30141705,noise,1
8,30141705,studio,1
9,51496964,comfort,1


In [347]:
# Count occurrences of each topic
topic_counts = exploded_topics['topics'].value_counts().reset_index()

# Rename columns for clarity
topic_counts.columns = ['topic', 'count']

# Sort by count descending to identify the main topics
sorted_topic_counts = topic_counts.sort_values(by='count', ascending=False).head(5)

print("Most common topics in negative comments:")
print(sorted_topic_counts)

Most common topics in negative comments:
                   topic  count
0               location     13
2            cleanliness      5
1                  noise      5
3  sleeping arrangements      3
4                comfort      3


#### Properties with the highest number of negative reviews

In [348]:
# Group by listing_id and count occurrences
listing_negative_counts = negative_comments['listing_id'].value_counts().reset_index()

# Rename columns for clarity
listing_negative_counts.columns = ['listing_id', 'negative_review_count']

# Sort by negative review count descending to identify listings with the most negative reviews
sorted_listing_negative_counts = listing_negative_counts.sort_values(by='negative_review_count', ascending=False)

print("Listings with the highest number of negative reviews:")
print(sorted_listing_negative_counts)

Listings with the highest number of negative reviews:
            listing_id  negative_review_count
0   872950232509198703                      2
1             53354926                      2
2             20399917                      2
3   962127121469363434                      2
4   971811518060382945                      2
5   935332067081342415                      2
6             25175483                      2
7   959893956899055246                      2
8             51496964                      1
9             30141705                      1
10  943817677972185100                      1


#### Get all the information on the top 5 properties

In [362]:
# Convert listing_id to string if necessary
top_5_properties['listing_id'] = top_5_properties['listing_id'].astype(str)

# Since 1 of the url's doesnt work
top_4_properties = top_5_properties[top_5_properties['listing_id'] != '962127121469363434']

# Filter df_listings based on the top_properties listing_id
selected_listings = df_listings[df_listings['id'].astype(str).isin(top_4_properties['listing_id'])]

print("Selected listings:")
selected_listings.to_csv('Top4propertiesListings.csv', index=False)
print(selected_listings)


Selected listings:
                     id                                      listing_url  \
93   959893956899055246  https://www.airbnb.com/rooms/959893956899055246   
120  877994002042801699  https://www.airbnb.com/rooms/877994002042801699   
625            30141705            https://www.airbnb.com/rooms/30141705   
658  872950232509198703  https://www.airbnb.com/rooms/872950232509198703   

                                                  name  \
93                 trafalgar sq 1 bedr/3 beds sleeps 6   
120  bright spacious 2 bedrooms apartment in bayswater   
625                          elegant kensington studio   
658   luxstay covent garden apartment - sleeps up to 8   

                                           description    host_id  \
93   large apartment in covent garden <br />5 mins ...    6717859   
120  thanks to the central location of this accommo...    9684327   
625  stunning studio located on the ground floor of...  226472167   
658  enjoy a stylish experience a

In [363]:
# Filter df_reviews based on the top_properties listing_id
selected_reviews = df_reviews[df_reviews['listing_id'].astype(str).isin(top_4_properties['listing_id'])]

print("Selected reviews:")
selected_reviews.to_csv('Top4propertiesReviews.csv', index=False)
display(selected_reviews)

Selected reviews:


,listing_id,num_comments,id,date,reviewer_id,reviewer_name,comments,topics,sentiment
25,872950232509198703,24,1053558874453643417,2023-12-24,124348616,Apoorv,superb place for the Christmas festivities tha...,"christmas festivities, location, 5 star",positive
26,872950232509198703,24,1054964040355363780,2023-12-26,28739966,Simona,È’ stato uno stupendo soggiorno nei giorni di ...,"cozy, central, responsive",positive
27,872950232509198703,24,1057943467152600354,2023-12-30,539297323,Joseph,Nikko was an ideal host. Checking in and out w...,"family friendly, ideal location, christmas vac...",positive
28,872950232509198703,24,1060020261795206005,2024-01-02,416235243,Radoy,Nice place!,"nice place, family friendly",positive
29,872950232509198703,24,1062192762010463340,2024-01-05,27832972,Pierre-Frédéric,Superbe emplacement pour visiter Londres en fa...,"family friendly, location, responsive host",positive
...,...,...,...,...,...,...,...,...,...
365,959893956899055246,15,1095645498070774307,2024-02-20,24459590,Rai,I’ve stayed in London several times in the pas...,"location, apartment, host",positive
366,959893956899055246,15,1099203608918324845,2024-02-25,62983740,Gabrielle,Steve’s place was just what we needed. The loc...,"location, accessibility, entertainment",positive
367,959893956899055246,15,1101409329332586088,2024-02-28,218420134,Jessica,Another excellent stay. This has fast become m...,"excellent stay, favorite airbnb, london",positive
368,959893956899055246,15,1110054180139338227,2024-03-11,501698457,Sam,Perfect location and great hosts. Thank you!,"location, hosts, perfect",positive


In [368]:
# Split the 'topics' column into lists of words
positive_comments['topics'] = positive_comments['topics'].str.split(', ')

# Explode the lists into individual rows
exploded_topics = positive_comments.explode('topics')

# Save all the words in a new DataFrame
words_df = exploded_topics[['topics']].reset_index(drop=True)

display(words_df)


C:\Users\MartaMoreira\AppData\Local\Temp\ipykernel_18368\3421902150.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  positive_comments['topics'] = positive_comments['topics'].str.split(', ')


,topics
0,location
1,cleanliness
2,amenities
3,clean
4,comfortable
...,...
899,hosts
900,perfect
901,location
902,cleanliness


In [370]:
words_df.to_csv('wordsTop4properties.csv', index=False)

#### Average price per night on top 20 properties

In [367]:
# Merge top_20_properties with df_listings on listing_id and id
merged_listings = pd.merge(top_20_properties, df_listings, left_on='listing_id', right_on='id', how='inner')

# Calculate average price per night for the top 20 properties
average_price_per_night = merged_listings['price'].mean()


print("\nAverage price per night for the top 20 properties:")
print(average_price_per_night)


Average price per night for the top 20 properties:
178.65
